In [17]:
import requests
import pandas as pd
import os

In [18]:
# future warningを非表示
import warnings
warnings.simplefilter('ignore', FutureWarning)

In [19]:
# 定数を記載
REQUEST_URL = 'https://app.rakuten.co.jp/services/api/Travel/SimpleHotelSearch/20170426'
APP_ID = '**' # 非表示

In [20]:
# パラメータを辞書型で入力
params = {
    'format': 'json',
    'largeClassCode': 'japan',
    'middleClassCode' : 'kyoto',
    'smallClassCode' : 'shi',
    'detailClassCode' : 'A',
    'page' : '1',
    'datumType' : '2',
    'applicationId' : APP_ID
}

In [21]:
res = requests.get(REQUEST_URL, params)

In [22]:
# res 200 は　レスポンスに成功した　という意味。エラーの場合はAPIサイトで数字でエラーを判別
res

<Response [200]>

In [23]:
result = res.json()  # レスポンスの中身を表示

pageCount: 総ページ数<br>
page: 1ページ目だけ取ってきている<br>
first: 1page目の1件目<br>

In [24]:
result  # 大きく分けて、「pagingInfo」「hotels」に分かれている

{'pagingInfo': {'recordCount': 88,
  'pageCount': 3,
  'page': 1,
  'first': 1,
  'last': 30},
 'hotels': [{'hotel': [{'hotelBasicInfo': {'hotelNo': 130702,
      'hotelName': '京都\u3000嵐山温泉\u3000花伝抄（共立リゾート）',
      'hotelInformationUrl': 'https://img.travel.rakuten.co.jp/image/tr/api/hs/dQ4dX/?f_no=130702',
      'planListUrl': 'https://img.travel.rakuten.co.jp/image/tr/api/hs/cHNRi/?f_no=130702&f_flg=PLAN',
      'dpPlanListUrl': 'https://img.travel.rakuten.co.jp/image/tr/api/hs/TDZXm/?noTomariHotel=130702',
      'reviewUrl': 'https://img.travel.rakuten.co.jp/image/tr/api/hs/RmfmX/?f_hotel_no=130702',
      'hotelKanaName': 'きょうとあらしやまおんせん\u3000かでんしょう',
      'hotelSpecial': '〜安心安全★コロナ対策実施中〜楽天アワード受賞【４年連続京都旅館カテゴリー宿泊数NO.1】',
      'hotelMinCharge': 12000,
      'latitude': 126026.31,
      'longitude': 488461.0,
      'postalCode': '616-0003',
      'address1': '京都府',
      'address2': '京都市西京区嵐山西一川町5-4',
      'telephoneNo': '075-863-0489',
      'faxNo': '075-863-5780',
      'access':

In [25]:
# hotel_info = result['hotels'][0]['hotel'][0]['hotelBasicInfo']
# hotel_info

In [26]:
rating_info = result['hotels'][2]['hotel'][1]['hotelRatingInfo']
pd.DataFrame(rating_info, index=[2])

,serviceAverage,locationAverage,roomAverage,equipmentAverage,bathAverage,mealAverage
2,3.88,3.79,3.81,3.91,4.4,3.72


In [27]:
hotel_info = result['hotels'][2]['hotel'][0]['hotelBasicInfo']
pd.DataFrame(hotel_info, index=[2])

,hotelNo,hotelName,hotelInformationUrl,planListUrl,dpPlanListUrl,reviewUrl,hotelKanaName,hotelSpecial,hotelMinCharge,latitude,...,parkingInformation,nearestStation,hotelImageUrl,hotelThumbnailUrl,roomImageUrl,roomThumbnailUrl,hotelMapImageUrl,reviewCount,reviewAverage,userReview
2,10854,京都竹の郷温泉 万葉の湯 ホテル京都エミナース,https://img.travel.rakuten.co.jp/image/tr/api/...,https://img.travel.rakuten.co.jp/image/tr/api/...,https://img.travel.rakuten.co.jp/image/tr/api/...,https://img.travel.rakuten.co.jp/image/tr/api/...,きょうとたけのさとおんせん まんようのゆ ほてるきょうとえみなーす,ご宿泊者は24時間万葉の湯入り放題！.心ゆくまで温泉を満喫下さい。,11400,125861.77,...,有り 250台 無料,桂,https://img.travel.rakuten.co.jp/share/HOTEL/1...,https://img.travel.rakuten.co.jp/HIMG/90/10854...,https://img.travel.rakuten.co.jp/share/HOTEL/1...,https://img.travel.rakuten.co.jp/HIMG/INTERIOR...,https://img.travel.rakuten.co.jp/share/HOTEL/1...,485,3.97,駐車場は屋内にありフロントの対応も満足いたしましたウオシュレットが故障してましたのが残念てす...


In [28]:
result['hotels'][2]['hotel'][1]['hotelRatingInfo']

{'serviceAverage': 3.88,
 'locationAverage': 3.79,
 'roomAverage': 3.81,
 'equipmentAverage': 3.91,
 'bathAverage': 4.4,
 'mealAverage': 3.72}

In [ ]:
# 下記の処理の繰り返し
for page in range(1, 26):
    params = {
        'format': 'json',
        'largeClassCode': 'japan',
        'middleClassCode' : 'kyoto',
        'smallClassCode' : 'shi',
        'detailClassCode' : 'D',
        'page' : str(page),
        'datumType' : '1',
        'applicationId' : APP_ID
    }

    res = requests.get(REQUEST_URL, params)
    result = res.json()

    df = pd.DataFrame()
    hotels = result['hotels']

    for i, hotel in enumerate(hotels):
        hotel_info = hotel['hotel'][0]['hotelBasicInfo']
        rating_info = result['hotels'][i]['hotel'][1]['hotelRatingInfo']
        _df1 = pd.DataFrame(hotel_info, index=[i])
        _df2 = pd.DataFrame(rating_info, index=[i])
        _df = pd.concat([_df1, _df2], axis=1)
        df = df.append(_df)

    df[['hotelNo', 'hotelName', 'hotelInformationUrl', 'hotelKanaName', 'hotelMinCharge', 
        'latitude', 'longitude', 'postalCode', 'address1','address2', 'telephoneNo', 'access', 'parkingInformation',
       'nearestStation', 'reviewCount', 'reviewAverage', 'serviceAverage', 'locationAverage', 'roomAverage',	
        'equipmentAverage',	'bathAverage', 'mealAverage', 'userReview']].to_csv(f'hoteldata/D/hotel_d{page}.csv', index=False)


In [ ]:
# CSVファイルが保存されているディレクトリのパス
path = 'hoteldata/A/'

# ディレクトリ内の全CSVファイル名のリストを作成
all_files = [os.path.join(path, f'hotel_a{i}.csv') for i in range(1, 4)]

# 全CSVファイルを読み込んで結合
df_from_each_file = (pd.read_csv(f) for f in all_files)
df_merged   = pd.concat(df_from_each_file, ignore_index=True)

# 結合したデータを新しいCSVファイルに出力
df_merged.to_csv('hoteldata/hotel_a.csv', index=False)


In [31]:
hotel_e = pd.read_csv('hoteldata/hotel_e.csv')
hotel_e['category'] = 'E'
hotel_e.to_csv('hoteldata/hotel_e.csv')

In [32]:
# CSVファイルが保存されているディレクトリのパス
path = 'hoteldata'

# 全てのCSVファイル名
files = ['hotel_a.csv', 'hotel_b.csv', 'hotel_c.csv', 'hotel_d.csv', 'hotel_e.csv']

# 全CSVファイルのパスを取得
all_files = [os.path.join(path, f) for f in files]

# 全CSVファイルを読み込んで結合
df_from_each_file = (pd.read_csv(f) for f in all_files)
df_merged   = pd.concat(df_from_each_file, ignore_index=True)

# 結合したデータを新しいCSVファイルに出力
df_merged.to_csv('hoteldata/hotel_total.csv', index=False)